In [28]:
import torch
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [29]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, 120)
        self.fc2 = nn.Linear(120, 64)
        self.fc3 = nn.Linear(64, 3)

    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        x = self.fc3(x)

        return x

In [30]:
data = pd.read_csv('datasets/dataset_completo.csv')
x = data.iloc[:, :719].values.astype(dtype=np.float32)
y = data.iloc[:, 720].values.astype(dtype=np.int64) 
feature_index = list(range(0,719))
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [38]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

select_k_best = SelectKBest(mutual_info_classif, k=10)
X_train_k_best = select_k_best.fit_transform(X_train_scaled, y_train)
X_test_k_best = select_k_best.transform(X_test_scaled)

selected_features_indices = select_k_best.get_support(indices=True)

np.savetxt('datasets/selected_features.csv', selected_features_indices, delimiter=',', fmt='%d')

[586, 591, 592, 593, 622, 623, 624, 625, 627, 630]


In [32]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

xs_train = torch.tensor(X_train_k_best).view(-1, len(X_train_k_best[0])).to(device)
ys_train = torch.tensor(y_train).to(device)
xs_test = torch.tensor(X_test_k_best).view(-1, len(X_test_k_best[0])).to(device)
ys_test = torch.tensor(y_test).to(device)

In [33]:
mean = torch.mean(xs_train, dim=0)
std_dev = torch.std(xs_train, dim=0)
mean_broadcasted = mean.unsqueeze(0).expand_as(xs_train)
std_dev_broadcasted = std_dev.unsqueeze(0).expand_as(xs_train)
xs_train_z_score = (xs_train - mean_broadcasted) / std_dev_broadcasted
mean_broadcasted = mean.unsqueeze(0).expand_as(xs_test)
std_dev_broadcasted = std_dev.unsqueeze(0).expand_as(xs_test)
xs_test_z_score = (xs_test - mean_broadcasted) / std_dev_broadcasted

In [ ]:
max_value = 12
min_value = 0

xs_train_min_max = (xs_train - min_value)/(max_value - min_value)
xs_test_min_max = (xs_test - min_value)/(max_value - min_value)

In [34]:
model = NeuralNetwork(len(xs_train_z_score[0]))
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

batch_size = 100
torch.manual_seed(42)
dataset = TensorDataset(xs_train_z_score,ys_train)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Train the model
num_epochs = 2000
for epoch in range(num_epochs):
    for batch_idx, (xs_batch, ys_batch) in enumerate(dataloader):
        outputs = model(xs_batch)
        loss = criterion(outputs, ys_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if(epoch % 100 == 0):
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

print('Training finished.')


Epoch [1/2000], Loss: 0.2691667973995209
Epoch [101/2000], Loss: 8.246210200013593e-05
Epoch [201/2000], Loss: 2.5873947379295714e-05
Epoch [301/2000], Loss: 1.4322539755085018e-05
Epoch [401/2000], Loss: 6.882438810862368e-06
Epoch [501/2000], Loss: 1.3443061561702052e-06
Epoch [601/2000], Loss: 4.73165812309162e-07
Epoch [701/2000], Loss: 2.0631994175346335e-06
Epoch [801/2000], Loss: 1.6139081537858146e-07
Epoch [901/2000], Loss: 1.1737520111410049e-07
Epoch [1001/2000], Loss: 7.335955842791009e-09
Epoch [1101/2000], Loss: 2.384184938364342e-08
Epoch [1201/2000], Loss: 1.8339889606977522e-09
Epoch [1301/2000], Loss: 0.0
Epoch [1401/2000], Loss: 0.0
Epoch [1501/2000], Loss: 0.0
Epoch [1601/2000], Loss: 0.0
Epoch [1701/2000], Loss: 0.0
Epoch [1801/2000], Loss: 0.0
Epoch [1901/2000], Loss: 0.0
Training finished.


In [35]:
with torch.no_grad():
    outputs = model(xs_test_z_score)
# Metrics calculation (optional)
_, predicted = torch.max(outputs, 1)
accuracy = (predicted == ys_test).sum().item() / ys_test.size(0)
# Calculate other metrics as needed

print(f"Loss: {loss.item()}")
print(f"Accuracy: {accuracy}")

Loss: 0.0
Accuracy: 0.9976019184652278


In [37]:
model.eval()
with torch.no_grad():
    predicted_value = model(torch.tensor(xs_test[0]).to(device))
print("Input Feature: " + str(xs_test[0]))
print("Expected Value: " + str(ys_test[0]))
print("Model Output: " + str(predicted_value))
print("Model Prediction: " + str(torch.argmax(predicted_value)))

Input Feature: tensor([-0.7100, -1.9577, -1.1945, -1.1721, -0.9902, -0.9851, -0.9834, -0.9904,
        -1.0226, -1.1123])
Expected Value: tensor(1)
Model Output: tensor([-40.9136,  -4.0822, -32.6478])
Model Prediction: tensor(1)


C:\Users\gsoch\AppData\Local\Temp\ipykernel_7860\2318861247.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  predicted_value = model(torch.tensor(xs_test[0]).to(device))


In [20]:
torch.save(model.state_dict(), 'datasets/model1.pth')

In [21]:
model = NeuralNetwork(len(xs_train_z_score[0]))
model.load_state_dict(torch.load('datasets/model1.pth'))
model.eval()

NeuralNetwork(
  (fc1): Linear(in_features=10, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=3, bias=True)
)

In [24]:
with torch.no_grad():
    predicted_value = model(torch.tensor(xs_test[400]).to(device))
print("Input Feature: " + str(xs_test[400]))
print("Expected Value: " + str(ys_test[400]))
print("Model Prediction: " + str(torch.argmax(predicted_value)))

Input Feature: tensor([-1.0493, -1.2094, -1.3192, -1.3859, -1.0902, -1.0788, -1.5170, -1.0826,
        -1.0960, -0.9731])
Expected Value: tensor(1)
Model Prediction: tensor(1)


C:\Users\gsoch\AppData\Local\Temp\ipykernel_7860\1364314063.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  predicted_value = model(torch.tensor(xs_test[400]).to(device))
